## Import Necessary Libraries

In [1]:
import pandas as pd

from datetime import datetime
import time

## Import, Inspect, and Clean First Dataset - Spotify Track Data

The first csv, Hot 100 Audio Features, contains data from spotify on tracks that have at all been on the Billboard Hot 100 at any time between August 1958 and May 2021.

In [2]:
# Use pd.read_csv() to read in first dataset - Hot 100 Audio Features
audio_features = pd.read_csv('Resources/Hot 100 Audio Features.csv').drop(columns='index')
audio_features.head(2)

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,-twistin'-White Silver SandsBill Black's Combo,Bill Black's Combo,-twistin'-White Silver Sands,[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,¿Dònde Està Santa Claus? (Where Is Santa Claus...,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),['novelty'],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Drop and filter out unneeded data

In [3]:
# Select only those rows that have an actual genre in them; none of the rows that have only [] in the genre column
audio_features = audio_features[audio_features['spotify_genre'] != '[]']

In [4]:
# Drop unneeded columns
audio_features = audio_features.drop(columns=["spotify_track_id", "spotify_track_preview_url", "spotify_track_album", "spotify_track_popularity"])

In [5]:
# Rename some columns for easier use
audio_features = audio_features.rename(columns={"spotify_track_duration_ms":"duration_ms", "spotify_track_explicit":"explicit"})

In [6]:
# Inspect data for null values and data types
audio_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26946 entries, 1 to 29502
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SongID            26946 non-null  object 
 1   Performer         26946 non-null  object 
 2   Song              26946 non-null  object 
 3   spotify_genre     25346 non-null  object 
 4   duration_ms       22936 non-null  float64
 5   explicit          22936 non-null  object 
 6   danceability      22883 non-null  float64
 7   energy            22883 non-null  float64
 8   key               22883 non-null  float64
 9   loudness          22883 non-null  float64
 10  mode              22883 non-null  float64
 11  speechiness       22883 non-null  float64
 12  acousticness      22883 non-null  float64
 13  instrumentalness  22883 non-null  float64
 14  liveness          22883 non-null  float64
 15  valence           22883 non-null  float64
 16  tempo             22883 non-null  float64
 17

In [7]:
# Drop all null values
audio_features = audio_features.dropna().reset_index(drop=True)

# Check that null values were dropped correctly
audio_features.isna().sum()

SongID              0
Performer           0
Song                0
spotify_genre       0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64

#### Format the SongID, Performer, and Song column

In [8]:
# Use .str.title() to make sure all SongIDs, Performers, and Songs are formatted the same
audio_features["SongID"]= audio_features["SongID"].str.title()
audio_features["Performer"]= audio_features["Performer"].str.upper().str.title()
audio_features["Song"]= audio_features["Song"].str.upper().str.title()

#### Clean the genre column

In [9]:
# Start cleaning the 'spotify_genre' column - use .replace() to drop brackets and quote marks; create a new column for cleaned data
for i, row in audio_features.iterrows():
    cleaning_genre = str(row['spotify_genre']).replace('[', '').replace(']', '').replace("'", "")
    audio_features.loc[i, 'genre'] = cleaning_genre

In [10]:
# Create a new row for each genre a song is classified under
audio_features['genre'] = audio_features['genre'].str.split(",")
audio_features = audio_features.explode('genre')

In [11]:
# Strip the rows in the 'genre' column - any genres after a comma in the original 'spotify_genre' column have a space at the beginning
# of the string
audio_features['genre'] = audio_features['genre'].str.strip()

Import the parent_genres csv to reclassify song genres.

In [12]:
# Import the parent_genres csv to match genres with their parent genre
parent_genres = pd.read_csv('Resources/parent_genres.csv')

In [13]:
# Merge audio_features and parent_genres on the 'genre' column and drop the 'spotify_genre' and 'genre' columns
audio_features = audio_features.merge(parent_genres, on="genre", how="left").drop(columns=['spotify_genre', 'genre'])

# Rename the new 'parent_genre' column as 'genre'
audio_features = audio_features.rename(columns={'parent genre':'genre'})

#### Drop duplicates

In [14]:
# Drop all duplicate Song IDs
audio_features = audio_features.drop_duplicates(subset="SongID").reset_index(drop=True)

#### Add a new column

In [15]:
# Create a new column with track duration in minutes calculated
audio_features['duration_m'] = audio_features['duration_ms']/60000

#### Review final DataFrame

In [16]:
# Final inspection of audio_features dataframe
audio_features.head()

,SongID,Performer,Song,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,genre,duration_m
0,......And Roses And Rosesandy Williams,Andy Williams,......And Roses And Roses,166106.0,False,0.154,0.185,5.0,-14.063,1.0,0.0315,0.91100,0.000267,0.112,0.150,83.969,4.0,adult standards,2.768433
1,...And Then There Were Drumssandy Nelson,Sandy Nelson,...And Then There Were Drums,172066.0,False,0.588,0.672,11.0,-17.278,0.0,0.0361,0.00256,0.745000,0.145,0.801,121.962,4.0,rock,2.867767
2,...Baby One More Timebritney Spears,Britney Spears,...Baby One More Time,211066.0,False,0.759,0.699,0.0,-5.745,0.0,0.0307,0.20200,0.000131,0.443,0.907,92.960,4.0,pop,3.517767
3,...Ready For It?Taylor Swift,Taylor Swift,...Ready For It?,208186.0,False,0.613,0.764,2.0,-6.509,1.0,0.1360,0.05270,0.000000,0.197,0.417,160.015,4.0,pop,3.469767
4,'65 Love Affairpaul Davis,Paul Davis,'65 Love Affair,219813.0,False,0.647,0.686,2.0,-4.247,0.0,0.0274,0.43200,0.000006,0.133,0.952,155.697,4.0,rock,3.663550


## Import, Inspect, and Clean Second Dataset - Billboard Hot 100 Data

The second dataset being imported is titled 'Hot Stuff' and it contains data on the ranking of all tracks in the Billboard Hot 100 each week from the week of August 2nd, 1958 to the week of May 29th, 2021.

In [17]:
# Use pd.read_csv() to read in second dataset - Hot Stuff
billboard_rankings = pd.read_csv('Resources/Hot Stuff.csv').drop(columns=['index', 'url', 'Previous Week Position'])
billboard_rankings.head()

,WeekID,Week Position,Song,Performer,SongID,Instance,Peak Position,Weeks on Chart
0,7/17/1965,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34,4
1,7/24/1965,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,22,5
2,7/31/1965,14,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,14,6
3,8/7/1965,10,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,10,7
4,8/14/1965,8,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,8,8


#### Format SongId, Performer, and Song column

In [18]:
# Use .str.title() to format all SongIDs, Performers, and Songs the same way
billboard_rankings["SongID"]= billboard_rankings["SongID"].str.upper().str.title()
billboard_rankings["Performer"]= billboard_rankings["Performer"].str.upper().str.title()
billboard_rankings["Song"]= billboard_rankings["Song"].str.upper().str.title()

#### Split WeekID column & find peak dates for each song

In [19]:
# Split the 'WeekID' column on the / characters to separate month, day, and year into their own columns
billboard_rankings[['Month', 'Day', 'Year']] = billboard_rankings['WeekID'].str.split('/', expand=True)

In [20]:
# Group the DataFrame by SongID and Year to find their highest position on the Billboard charts for each year it's in the charts
yearly_top = pd.DataFrame(billboard_rankings.groupby(['SongID', 'Year'])['Week Position'].min()).reset_index()
yearly_top = yearly_top.rename(columns={'Week Position':'Year Peak'})

# Group the DataFrame by SongID to find their highest position on the Billboard charts overall
billboard_top = pd.DataFrame(billboard_rankings.groupby(['SongID'])['Week Position'].min()).reset_index()
billboard_top = billboard_top.rename(columns={'Week Position':'Overall Peak'})

# Merge the 'Year Peak' and 'Overall Peak' column with the billboard_peaks DataFrame
billboard_peaks = billboard_rankings.merge(yearly_top, on=['SongID', 'Year'])
billboard_peaks = billboard_peaks.merge(billboard_top, on=['SongID'])

#### Calculate the maximum of the Weeks on Chart and Total Instances for each song

In [21]:
# Find the total number of weeks each song was on the Billboard charts
weeks_on_chart = pd.DataFrame(billboard_rankings.groupby(['SongID'])['Weeks on Chart'].max()).reset_index()
weeks_on_chart = weeks_on_chart.rename(columns={'Weeks on Chart':'Total Weeks on Chart'})

# Find the total number of instances each song was on the Billboard charts
total_instances = pd.DataFrame(billboard_rankings.groupby(['SongID'])['Instance'].max()).reset_index()
total_instances = total_instances.rename(columns={'Instance':'Total Instances'})

# Merge the 'Total Weeks on Chart' and 'Total Instances' columns with the billboard_peaks DataFrame
billboard_peaks = billboard_peaks.merge(weeks_on_chart, on=['SongID'])
billboard_peaks = billboard_peaks.merge(total_instances, on=['SongID'])

In [22]:
# Find the date that each song hit it's highest posittion on the Billboard charts (for each year it was on the chart) by finding the row
# where a song's Week Position matches it's Overall Peak position
for i, row in billboard_peaks.iterrows():
    if row['Week Position'] == row['Year Peak']:
        billboard_peaks.loc[i, 'initial_peak_yearly'] = row['WeekID']

#### Drop extra and duplicate rows

In [23]:
# Drop all rows with null values in the 'initial_peak_yearly' column
billboard_peaks = billboard_peaks.dropna(subset='initial_peak_yearly')

In [24]:
# Drop any duplicates based on SongID and Year - in case a song peaked at the same position more than once in the same year
# Keep the first row - the initial date they peaked that year
billboard_peaks = billboard_peaks.drop_duplicates(subset=['SongID', 'Year'])

### Create fully_cleaned_yearly dataframe (for yearly analysis on Tableau Dashboard) and save as csv

In [25]:
# Create a new dataframe to hold only the needed columns
billboard_peaks_yearly = billboard_peaks[['Song', 'Performer', 'SongID', 'Month', 'Day', 'Year', 'Year Peak', 'Overall Peak']]

# Merge the new dataframe with the audio_features dataframe to create the fully cleaned dataframe for yearly analysis
fully_cleaned_yearly = audio_features.merge(billboard_peaks_yearly, on=["Song", "Performer", "SongID"])
fully_cleaned_yearly = fully_cleaned_yearly.rename(columns={"spotify_track_duration_ms":"duration_ms", "spotify_track_explicit":"explicit",
                                                            "spotify_track_popularity":"spotify_popularity"})

In [26]:
fully_cleaned_yearly.to_csv('Resources/fully_cleaned_yearly.csv')

### Continue working with billboard_peaks dataframe to make cleaned_data dataframe for non-yearly analysis

#### Drop extra columns

In [27]:
# Use .drop to drop the 'Year Peak' and 'initial_peak_yearly' columns
billboard_peaks = billboard_peaks.drop(columns=['Year Peak', 'initial_peak_yearly'])

#### Find the first year each song reached it's highest position on the charts

In [28]:
# For songs that enter the chart in more than one year: find the year that each song hit it's highest posittion on the Billboard charts
# by finding the row where a song's Week Position matches it's Overall Peak position
for i, row in billboard_peaks.iterrows():
    if row['Week Position'] == row['Overall Peak']:
        billboard_peaks.loc[i, 'Peak Year'] = row['Year']

#### Drop extra and duplicate rows; drop unneeded columns

In [29]:
# Create a new dataframe to contain our clean data - drop all rows with a null value in the 'Peak Year' column
billboard_clean = billboard_peaks.dropna(subset='Peak Year')

In [30]:
# Drop any duplicates based on SongID - in case a song peaked at the same position more than once
billboard_clean = billboard_clean.drop_duplicates(subset=['SongID'])

In [31]:
# Drop all columns unnecessary for analysis and model building
billboard_clean = billboard_clean.drop(columns=['WeekID', 'Week Position', 'Instance', 'Peak Position', 'Weeks on Chart', 'Peak Year'])

#### Create a boolean column to indicate whether a song made it into the top 20 or not

In [32]:
# Loop through the DataFrame to create a new column, 'Top 20', that displays whether a song reached the top 20 (1) or didn't (0)
for i, row in billboard_clean.iterrows():
    if row['Overall Peak'] <= 20:
        billboard_clean.loc[i, 'Top 20'] = 1
    else:
        billboard_clean.loc[i, 'Top 20'] = 0

In [33]:
# Convert the column to data type integer
billboard_clean['Top 20'] = billboard_clean['Top 20'].astype(int)

In [34]:
# Reset the index
billboard_clean = billboard_clean.reset_index(drop=True)

#### Review final DataFrame

In [35]:
# Final check of the cleaned data
billboard_clean.head()

,Song,Performer,SongID,Month,Day,Year,Overall Peak,Total Weeks on Chart,Total Instances,Top 20
0,Don'T Just Stand There,Patty Duke,Don'T Just Stand Therepatty Duke,8,14,1965,8,11,1,1
1,Don'T Keep Wasting My Time,Teddy Pendergrass,Don'T Keep Wasting My Timeteddy Pendergrass,4,26,1997,90,6,1,0
2,Don'T Knock My Love - Pt. 1,Wilson Pickett,Don'T Knock My Love - Pt. 1Wilson Pickett,6,26,1971,13,12,1,1
3,Don'T Knock My Love,Diana Ross & Marvin Gaye,Don'T Knock My Lovediana Ross & Marvin Gaye,8,17,1974,46,9,1,0
4,Don'T Know Much,Bill Medley,Don'T Know Muchbill Medley,4,11,1981,88,4,1,0


## Create Final DataFrame

#### Merge the two DataFrames and rename columns for easier use

In [36]:
# Merge the audio_features DataFrame with the billboard_clean DataFrame to create our full, cleaned DataFrame to use for our analysis
full_clean_df = audio_features.merge(billboard_clean, on=["Song", "Performer", "SongID"])
full_clean_df = full_clean_df.rename(columns={"spotify_track_duration_ms":"duration_ms", "spotify_track_explicit":"explicit",
                                                "spotify_track_popularity":"spotify_popularity"})

#### Drop featured artists from the Performer column

In [37]:
full_clean_df[['Performer', 'Feature']] = full_clean_df['Performer'].str.split(' Featuring', expand=True)
full_clean_df = full_clean_df.drop(columns=['Feature'])

In [38]:
full_clean_df = full_clean_df.drop(4886, axis='index')

#### Review final DataFrame

In [39]:
# Final inspection
full_clean_df.head(2)

,SongID,Performer,Song,duration_ms,explicit,danceability,energy,key,loudness,mode,...,time_signature,genre,duration_m,Month,Day,Year,Overall Peak,Total Weeks on Chart,Total Instances,Top 20
0,......And Roses And Rosesandy Williams,Andy Williams,......And Roses And Roses,166106.0,False,0.154,0.185,5.0,-14.063,1.0,...,4.0,adult standards,2.768433,5,8,1965,36,7,1,0
1,...And Then There Were Drumssandy Nelson,Sandy Nelson,...And Then There Were Drums,172066.0,False,0.588,0.672,11.0,-17.278,0.0,...,4.0,rock,2.867767,10,6,1962,65,4,1,0


#### Save final DataFrame

In [40]:
full_clean_df.to_csv('Resources/cleaned_data.csv', index=False)